## Data Acquistion Exercises

In [78]:
import env
import acquire
import pandas as pd

1. Use a an SQL Query containing datasets as a source from the iris data. Create a pandas dataframe, df_iris, from this data.

In [80]:
df = acquire.get_iris_data()

,species_id,measurement_id,sepal_length,sepal_width,petal_length,petal_width,species_name
0,1,1,5.1,3.5,1.4,0.2,setosa
1,1,2,4.9,3.0,1.4,0.2,setosa
2,1,3,4.7,3.2,1.3,0.2,setosa
3,1,4,4.6,3.1,1.5,0.2,setosa
4,1,5,5.0,3.6,1.4,0.2,setosa


- Print the first 3 rows

In [67]:
df.head(3)

,species_id,measurement_id,sepal_length,sepal_width,petal_length,petal_width,species_name
0,1,1,5.1,3.5,1.4,0.2,setosa
1,1,2,4.9,3.0,1.4,0.2,setosa
2,1,3,4.7,3.2,1.3,0.2,setosa


- Print the number of rows and columns (shape)

In [54]:
df.shape

(150, 7)

- Print the column names

In [55]:
df.columns

Index(['species_id', 'measurement_id', 'sepal_length', 'sepal_width',
       'petal_length', 'petal_width', 'species_name'],
      dtype='object')

- Print the data type of each column

In [56]:
df.dtypes

species_id          int64
measurement_id      int64
sepal_length      float64
sepal_width       float64
petal_length      float64
petal_width       float64
species_name       object
dtype: object

- Print the summary statistics for each of the numeric variables. Would you recommend rescaling the data based on these statistics?

In [58]:
df.describe()

,species_id,measurement_id,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000
mean,2.000000,75.500000,5.843333,3.057333,3.758000,1.199333
std,0.819232,43.445368,0.828066,0.435866,1.765298,0.762238
min,1.000000,1.000000,4.300000,2.000000,1.000000,0.100000
25%,1.000000,38.250000,5.100000,2.800000,1.600000,0.300000
50%,2.000000,75.500000,5.800000,3.000000,4.350000,1.300000
75%,3.000000,112.750000,6.400000,3.300000,5.100000,1.800000
max,3.000000,150.000000,7.900000,4.400000,6.900000,2.500000


** I would not recommend rescaling the data because the values are all on the same scale.

2. Read Table1_CustDetails the excel module dataset, Excel_Exercises.xlsx, into a dataframe, df_excel

In [62]:
df_excel = pd.read_excel('Excel_Exercises.xlsx', sheet_name=0)

- Assign the first 100 rows to a new dataframe, df_excel_sample

In [48]:
df_excel_sample = df_excel[:100]

- Print the number of rows of your original dataframe

In [49]:
len(df_excel.index)

7049

- Print the first 5 column names

In [51]:
list(df_excel.columns[:5])

['customer_id', 'gender', 'is_senior_citizen', 'partner', 'dependents']

- Print the column names that have a data type of object

In [72]:
num_cols = list(df_excel.dtypes[df_excel.dtypes=='object'].index)

- Compute the range for each of the numeric variables.

In [82]:
df_excel.describe()

,is_senior_citizen,phone_service,internet_service,contract_type,monthly_charges,total_charges
count,7049.000000,7049.000000,7049.000000,7049.000000,7049.000000,7038.000000
mean,0.162009,1.324585,1.222585,0.690878,64.747014,2283.043883
std,0.368485,0.642709,0.779068,0.833757,30.099460,2266.521984
min,0.000000,0.000000,0.000000,0.000000,18.250000,18.800000
25%,0.000000,1.000000,1.000000,0.000000,35.450000,401.587500
50%,0.000000,1.000000,1.000000,0.000000,70.350000,1397.100000
75%,0.000000,2.000000,2.000000,1.000000,89.850000,3793.775000
max,1.000000,2.000000,2.000000,2.000000,118.750000,8684.800000


3. Read the data from this google sheet into a dataframe, df_google

In [86]:
sheet_url = 'https://docs.google.com/spreadsheets/d/1Uhtml8KY19LILuZsrDtlsHHDC9wuDGUSe8LTEwvdI5g/edit#gid=341089357'

csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

df_googlesheet = pd.read_csv(csv_export_url)

- Print the first 3 rows

In [85]:
df_googlesheet.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


- Print the number of rows and columns

In [88]:
df_googlesheet.shape

(891, 12)

- Print the column names

In [92]:
df_googlesheet.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

- Print the data type of each column

In [96]:
df_googlesheet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


- Print the summary statistics for each of the numeric variables

In [ ]:
df_googlesheet.describe()

- Print the unique values for each of your categorical variables

In [95]:
for col in df_googlesheet:
    if df_googlesheet[col].dtype=='O':
       print(df_googlesheet[col].value_counts()[0:5])

Ibrahim Shawah, Mr. Yousseff                         1
Danoff, Mr. Yoto                                     1
McCoy, Miss. Agnes                                   1
Astor, Mrs. John Jacob (Madeleine Talmadge Force)    1
Dean, Master. Bertram Vere                           1
Name: Name, dtype: int64
male      577
female    314
Name: Sex, dtype: int64
CA. 2343    7
1601        7
347082      7
3101295     6
347088      6
Name: Ticket, dtype: int64
C23 C25 C27    4
G6             4
B96 B98        4
C22 C26        3
E101           3
Name: Cabin, dtype: int64
S    644
C    168
Q     77
Name: Embarked, dtype: int64


## Data Preperation Exercises

1. Iris Data

- Use the function defined in acquire.py to load the iris data.

- Drop the species_id and measurement_id columns.

- Rename the species_name column to just species.

- Encode the species name using a sklearn label encoder. Research the inverse_transform method of the label encoder. How might this be useful?

- Create a function named prep_iris that accepts the untransformed iris data, and returns the data with the transformations above applied.

2. Titanic Data

- Use the function you defined in acquire.py to load the titanic data set.

- Handle the missing values in the embark_town and embarked columns.

- Remove the deck column.

- Use a label encoder to transform the embarked column.

- Scale the age and fare columns using a min max scaler. Why might this be beneficial? When might you not want to do this?

- Create a function named prep_titanic that accepts the untransformed titanic data, and returns the data with the transformations above applied.